In [1]:
!pip install torchextractor

In [2]:
# %pip install torchmetrics[image]


In [3]:
import torch
from PIL import Image
import wandb
import torchmetrics
from torchvision.models import alexnet, inception_v3
import json
import os
from os.path import join as opj
from torchvision.transforms import *
import numpy as np
from scipy import signal
import tqdm
from torchsummary import summary
import torchextractor as tx
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import clip
from torchmetrics.image.fid import FrechetInceptionDistance
import pandas as pd

/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
indices=[25,31,68,121,126,318,384,492,531,606,702,860]

indices2=[70,116,165,261,278,363,451,774]
indices3=[41,205,230,411,428,446,502,777]
extra=[95,905]

In [5]:
# target_run="rural-blaze-28"
# target_run="warm-mountain-29"

target_run="resilient-pyramid-1"  #sub01
# target_run="legendary-violet-2"  #sub02




In [6]:
api=wandb.Api()
project=api.project("BrainCaptioning")

entity, project = "matteoferrante", "BrainCaptioning"  # set to your entity and project
runs = api.runs(entity + "/" + project)

In [7]:
active_run=None
for run in api.runs(project):
    if run.name==target_run:
        print(f"found {target_run}")
        active_run=target_run
        
        for artifact in run.logged_artifacts():
            artifact_path = artifact.download()
            
            if "-table:v0" in artifact_path:
                decoded_artifact=artifact_path
                print(artifact_path)
                f=open(f"{artifact_path}/table.table.json")
                meta = json.load(f)
                data=meta["data"]
                
                image_captions=[]
                brain_captions=[]
                init_guess=[]
                depth_guess=[]
                shown=[]
                generated=[]
                generated_control=[]
                for i in range(len(data)):
                    image_captions.append(data[i][0])
                    
                    brain_captions.append(data[i][1])
                    
                    shown.append(data[i][2]["path"])
                    
                    init_guess.append(data[i][3]["path"])
                    depth_guess.append(data[i][4]["path"])
                    
                    generated.append(data[i][5]["path"])
                    generated_control.append(data[i][7]["path"])
                    
                
            
        break
        

found resilient-pyramid-1


wandb:   27 of 27 files downloaded.  
wandb: Downloading large artifact run-s51yzgh9-table:v0, 3790.83MB. 12700 files... 
wandb:   12700 of 12700 files downloaded.  
Done. 0:0:9.4


./artifacts/run-s51yzgh9-table:v0


## Decoded images


In [8]:

shown_imgs=[Image.open(opj(decoded_artifact,i)) for i in shown]
generated_imgs=[Image.open(opj(decoded_artifact,i)) for i in generated]
generated_control_imgs=[Image.open(opj(decoded_artifact,i)) for i in generated_control]
# depth_guess_imgs=[Image.open(opj(decoded_artifact,i))for i in depth_guess]

# init_guess_imgs=[Image.open(opj(decoded_artifact,i))for i in init_guess]
# 
                    
# depth_guess_1_imgs=np.array(depth_guess)[indices]
# depth_guess_1_imgs=[Image.open(opj(decoded_artifact,i)) for i in depth_guess_1_imgs]


# init_guess_1_imgs=np.array(init_guess)[indices]
# init_guess_1_imgs=[Image.open(opj(decoded_artifact,i)) for i in init_guess_1_imgs]

# compare_brain_1_captions=np.array(brain_captions)[indices]
# compare_image_1_captions=np.array(image_captions)[indices]


# depth_guess_2_imgs=np.array(depth_guess)[indices2]
# depth_guess_2_imgs=[Image.open(opj(decoded_artifact,i)) for i in depth_guess_2_imgs]


# init_guess_2_imgs=np.array(init_guess)[indices2]
# init_guess_2_imgs=[Image.open(opj(decoded_artifact,i)) for i in init_guess_2_imgs]

# compare_brain_2_captions=np.array(brain_captions)[indices2]
# compare_image_2_captions=np.array(image_captions)[indices2]


## Pixel Correlation

In [9]:
base_transform=Compose([Resize(425),ToTensor()])

In [10]:
first_candidates=[i for i in generated_imgs]
#first_candidates=[i for i in generated_control_imgs]


In [11]:
pix_corr=[]
for x,y in tqdm.tqdm(list(zip(shown_imgs,first_candidates))):
    x=base_transform(x).numpy()
    y=base_transform(y).numpy()
    pc= np.abs(np.corrcoef(x.flatten(), y.flatten())[0][1])
    pix_corr.append(pc)

100%|██████████████████████████████████████████████████████████| 982/982 [00:31<00:00, 30.78it/s]


## SSIM

In [12]:
ssim_metric=torchmetrics.StructuralSimilarityIndexMeasure()

In [13]:
x_torch=[base_transform(i) for i in shown_imgs]
y_torch=[base_transform(i) for i in first_candidates]


In [14]:
x_torch=torch.stack(x_torch)
y_torch=torch.stack(y_torch)

In [15]:
ssim=ssim_metric(x_torch,y_torch)

## N-way 

In [16]:
def n_way_acc(x,y,model,n=2,device="cpu"):
    
    model.to(device)
    cosine=torch.nn.CosineSimilarity()
    acc=0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(x))):
            original_features=model(x[i].unsqueeze(0).to(device)).cpu()
            generated_features=model(y[i].unsqueeze(0).to(device)).cpu()

            rnd_idx=torch.randint(len(x_torch),(1,)).item()

            random_features=model(y[rnd_idx].unsqueeze(0).to(device)).cpu()

            gen_dist=cosine(original_features.reshape(len(original_features),-1),generated_features.reshape(len(generated_features),-1))
            rnd_dist=cosine(original_features.reshape(len(original_features),-1),random_features.reshape(len(random_features),-1))
            if gen_dist>=rnd_dist:
                acc+=1
    return acc/len(x)

### AlexNet (2) and (5), InceptionNet_V3

In [17]:
alexnet_model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
alexnet_model.eval()

Using cache found in /home/matteo/.cache/torch/hub/pytorch_vision_v0.10.0
/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
alexnet_transforms=Compose([transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
            
    
inception_transforms=Compose([transforms.Resize(342),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
            
    


In [19]:
x_torch_alexnet=torch.stack([alexnet_transforms(i) for i in shown_imgs])
y_torch_alexnet=torch.stack([alexnet_transforms(i) for i in first_candidates])

In [20]:
x_torch_inception=torch.stack([inception_transforms(i) for i in shown_imgs])
y_torch_inception=torch.stack([inception_transforms(i) for i in first_candidates])

In [21]:
modules_2=list(alexnet_model.children())[0][:3]
modules_5=list(alexnet_model.children())[0][:10]
alexnet_model_2=torch.nn.Sequential(*modules_2)
alexnet_model_5=torch.nn.Sequential(*modules_5)



In [22]:
inception_model=inception_v3(pretrained=True)

inception_model.fc = torch.nn.Identity()
inception_model.eval()
print()

/home/matteo/anaconda3/envs/braindiff/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
# modules_in=list(inception_model.children())[:-3]

# inception_model=torch.nn.Sequential(*modules_in)

In [24]:
alex_2=n_way_acc(x_torch_alexnet,y_torch_alexnet,alexnet_model_2,device="cuda:2")

100%|█████████████████████████████████████████████████████████| 982/982 [00:04<00:00, 229.33it/s]


In [25]:
alex_5=n_way_acc(x_torch_alexnet,y_torch_alexnet,alexnet_model_5,device="cuda:2")

100%|█████████████████████████████████████████████████████████| 982/982 [00:04<00:00, 244.16it/s]


In [26]:
alex_2,alex_5

(0.9134419551934827, 0.9755600814663951)

In [27]:
inception=n_way_acc(x_torch_inception,y_torch_inception,inception_model,device="cuda:2")

100%|██████████████████████████████████████████████████████████| 982/982 [00:32<00:00, 30.57it/s]


In [28]:
inception

0.8492871690427699

### CLIP

In [29]:
def n_way_acc_clip(x,y,model,n=2,device="cpu",preprocess=None):
    
    cosine=torch.nn.CosineSimilarity()
    acc=0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(x))):
            
            orig_image = preprocess(x[i]).unsqueeze(0).to(device)
            
            gen_image = preprocess(y[i]).unsqueeze(0).to(device)
            
            rnd_idx=torch.randint(len(x_torch),(1,)).item()
            rnd_image = preprocess(y[rnd_idx]).unsqueeze(0).to(device)
            
            
            
            original_features=model.encode_image(orig_image).cpu().double()
            generated_features=model.encode_image(gen_image).cpu().double()


            random_features=model.encode_image(rnd_image).cpu().double()


            gen_dist=cosine(original_features.reshape(len(original_features),-1),generated_features.reshape(len(generated_features),-1))
            rnd_dist=cosine(original_features.reshape(len(original_features),-1),random_features.reshape(len(random_features),-1))
            if gen_dist>=rnd_dist:
                acc+=1
    return acc/len(x)

In [30]:
clip_model, preprocess = clip.load("ViT-B/32", device="cuda:2")

In [31]:
clip_acc=n_way_acc_clip(shown_imgs,first_candidates,clip_model,device="cuda:2",preprocess=preprocess)

100%|██████████████████████████████████████████████████████████| 982/982 [00:35<00:00, 27.69it/s]


## FID

In [32]:
# %pip install torchmetrics[image]

In [33]:



fid = FrechetInceptionDistance(feature=64,normalize=True)

fid.update(x_torch_inception, real=True)
fid.update(y_torch_inception, real=False)
f64=fid.compute()

# fid_2048 = FrechetInceptionDistance(feature=2048,normalize=True)

# fid_2048.update(x_torch_inception, real=True)
# fid_2048.update(y_torch_inception, real=False)
# f2048=fid_2048.compute()


    

In [34]:
f64

tensor(16.0055)

## Final metrics

In [35]:
print(f"PixCorr: \t {np.mean(pix_corr):.4f}")
print(f"SSIM: \t\t {ssim:.4f}")
print(f"AlexNet (2): \t {alex_2:.4f}")
print(f"AlexNet (5): \t {alex_5:.4f}")
print(f"Incpeption: \t {inception:.4f}")
print(f"CLIP: \t\t {clip_acc:.4f}")
print(f"FID: \t\t {f64.item():.4f}")




PixCorr: 	 0.3503
SSIM: 		 0.3267
AlexNet (2): 	 0.9134
AlexNet (5): 	 0.9756
Incpeption: 	 0.8493
CLIP: 		 0.9155
FID: 		 16.0055


In [36]:
df = pd.DataFrame({
    "Metric": ["PixCorr", "SSIM", "AlexNet (2)", "AlexNet (5)", "Inception", "CLIP", "FID"],
    "Value": [np.mean(pix_corr), ssim, alex_2, alex_5, inception, clip_acc, f64.item()]
})

df["Value"] = df["Value"].apply(lambda x: f"{x:.4f}")

df.to_csv(f"image_metrics_{target_run}")